In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field


def fake_db_retrival():
    # Simulating a database retrieval of product details
    with open('product_details.txt', 'r') as file:
        products = file.read().strip()
    return products

llm = ChatGroq(model="gemma2-9b-it")

# Product data structure.
class Product(BaseModel):
    item_number: str 
    category: str
    description: str 
    price: int
    product_information: str


parser = JsonOutputParser(pydantic_object=Product)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{question}\n Product details: {product_details}",
    input_variables=[
    ("system", "You are a product catalog sisstant. Here are the product details : {product_details}\n Respond to the user's question about products based on product_details, and always repeat the user's question back before you respond."),
    ("user", "{question}")
],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


chain = prompt | llm | parser

question = "Get me in ear headphones details?"
chain.invoke({"question": question, "product_details": fake_db_retrival()})


{'item_number': '10023',
 'category': 'In-Ear Headphones',
 'description': 'JBL Wave 200TWS, JBL Deep Bass Sound, 20 Hours of Combined Playback, Either Ear, or Both, Calls and Voice Assistant with Touch Control, Comfort-fit, Black',
 'price': 9999,
 'product_information': 'Style: True Wireless Stick\nColour: BLACK\nBrand\tJBL\nColour\tBLACK\nEar placement\tIn Ear\nForm factor\tIn Ear\nNoise control\tActive Noise Cancellation'}